<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Цель: защитить данные клиентов страховой компании «Хоть потоп». 

Задачи:
- Разработать метод преобразования данных, чтобы по ниму было сложно восстановить персональную информацию.
- Обоснуйте корректность его работы.
- Необходимо защитить данные так, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. 

Дополнительно: Подбирать наилучшую модель не требуется.

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('/datasets/insurance.csv')

In [3]:
df.head(10)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
5,1,43.0,41000.0,2,1
6,1,39.0,39700.0,2,0
7,1,25.0,38600.0,4,0
8,1,36.0,49700.0,1,0
9,1,32.0,51700.0,1,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


Изменим тип данных в столбцах возрасти зарплата

In [5]:
df[['Возраст','Зарплата']] = df[['Возраст','Зарплата']].astype(int)

Проверим данные на явные дубликаты и определим их количество по отношению к общему количеству данных.

In [6]:
df.duplicated().sum() / len(df)

0.0306

Дубликаты составляют 3% от общего количества данных, можно убрать их.

In [7]:
df = df.drop_duplicates()

Проверим

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4847 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   Пол                4847 non-null   int64
 1   Возраст            4847 non-null   int64
 2   Зарплата           4847 non-null   int64
 3   Члены семьи        4847 non-null   int64
 4   Страховые выплаты  4847 non-null   int64
dtypes: int64(5)
memory usage: 227.2 KB


Промежуточные выводы:
1. Загружены данные.
2. Изменены типы данных признаков возраста и заработной пллаты.
3. Данные проверены на явные дубликаты. Выявленные дубликаты обраны из данных.
4. В данных необнаружены пропуски.

## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — обратимая матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

Заменим матрицу $X$ на матрицу $Z$:

$$Z = XP$$

где $P$ - обратимая матрица с некими значениями, на которую может быть умножена матрица $X$.


Заменим $X$ на $Z$ и вычислим, чему будет равено предсказание и вектор весов.


$$a_1 = Zw_1$$

$$w_1 = (Z^T Z)^{-1} Z^T y $$

Подставим значения

$$a_1 = Z (Z^T Z)^{-1} Z^T y $$

Заменим все $Z$ 

$$a_1 = XP ((XP)^T (XP))^{-1} (XP)^T y $$


Раскроем скобки в 2 этапа:
$$
a_1 = XP(P^T X^T XP)^{-1} (XP)^T y
$$
$$
a_1 = XPP^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^T y
$$
Т.к. $$A^{-1}A = E$$

Можем преобразовать уравнение:
$$
a_1 = X (X^T X)^{-1} X^T y = X w = a
$$


## Алгоритм преобразования

**Алгоритм**

1. Создание матрицы признаков.
2. Создание кодирующей обратимой матрицы размером равным количеству признаков.
3. Прверка матрицы на обратимость.
4. Получение матрицы преобразованных признаков.
5. Обучение модели на двух мтарицах.
6. Проверка качества моделей на исходных и измененённых данных посредством сравнения r2.

**Обоснование**

Для преобразования данных необходима обратимая невырожденная матрица. Матрица должна быть такой же размерности как исходная по количеству признаков для регрессии.

## Проверка алгоритма

In [9]:
RANDOM_STATE=12345

In [10]:
features=df.drop('Страховые выплаты', axis =1)
target = df['Страховые выплаты']

features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.25, random_state=RANDOM_STATE)

model = LinearRegression()
model.fit(features_train, target_train)
predicts = model.predict(features_test)
r2_1 = r2_score(target_test, predicts)
print('Исходные данные r2:', r2_1)

Исходные данные r2: 0.42307727615837565


Создаю случайную матрицу нужной размерности.

In [11]:
n= features.shape[1]

In [12]:
matrix_rand = np.random.normal(size=(n, n))

Поверка матрицы на невырожденность.

In [13]:
if np.linalg.det(matrix_rand) != 0:
    print('correct')

correct


Обучим модель на признаках умноженных на случайную матрицу

In [14]:
features_changed = features @ matrix_rand
features_train_ch, features_test_ch, target_train_ch, target_test_ch = train_test_split(
    features_changed, target, test_size=0.25, random_state=RANDOM_STATE)

In [15]:
model.fit(features_train_ch, target_train_ch)
predicts_ch = model.predict(features_test_ch)
r2_1 = r2_score(target_test_ch, predicts_ch)
print('Измененённые данные r2:', r2_1)

Измененённые данные r2: 0.4230772761583821


R2 на обоих моделях равны до 12 знаков после запятой.

## Выводы

1. Загружены данные.
2. Разработан алгоритм преобразования данных.
3. Изменена исходная матрица согласно алгоритму.
4. Проверен алгоритм изменения данных, проверена метрика R2 на изначальных и изменённых данных.